## Scraping by HTML from Miflasim Web-Page

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time
from datetime import date, timedelta

In [2]:
Kineret_levels_df = pd.DataFrame(columns=['Survey_Date','Kinneret_Level']) # Create Dataframe with 2 columns.

In [7]:
start_date = date(1969, 1, 1)
end_date = date(2021, 4, 1)
delta = timedelta(days=1) # for adding 1 day every running
i=0

datetime.timedelta(days=1)

In [88]:
while start_date <= end_date:
    url ="http://kineret.org.il/miflasim/?singledate=+"+str(start_date)+"&submit=+%D7%94%D7%A6%D7%92+%D7%90%D7%AA+%D7%94%D7%9E%D7%A4%D7%9C%D7%A1+#singledate"
    response1 = requests.get(url)
    while respone1.status_code != 200:
        print(f'[{response.status_code}]Something is wrong on {n_page}, restarting in 60 seconds...')
        time.sleep(60)
        response1 = requests.get(url)
    page_html = BeautifulSoup(response1.content,"html.parser") # Response the Page html content
    miflas_height=page_html.find("span",attrs={"class":"hp_miflas_height"}) # Response the kineret miflas height
    Kineret_levels_df = Kineret_levels_df.append({'Survey_Date': str(start_date),'Kinneret_Level': miflas_height.text}, ignore_index=True) # adding the value to the dataframe
    start_date += delta # adding 1 day
    i=i+1
    if(i>365): # saving the data in the CSV every 365 days
        i=0
        Kineret_levels_df.to_csv('Kineret_levels_Scrapping.csv', index=False)

In [89]:
Kineret_levels_df.to_csv('Kineret_levels_Scrapping.csv', index=False)

In [92]:
Kineret_levels_df.head()

,Survey_Date,Kinneret_Level
0,1969-01-01,-209.17
1,1969-01-02,-209.21
2,1969-01-03,-209.22
3,1969-01-04,-209.25
4,1969-01-05,-209.27


In [93]:
Kineret_levels_df.tail()

,Survey_Date,Kinneret_Level
19109,2021-04-27,-209.135
19110,2021-04-28,-209.145
19111,2021-04-29,-209.14
19112,2021-04-30,-209.145
19113,2021-05-01,-209.15


In [72]:
Kineret_levels_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3989 entries, 0 to 3988
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Survey_Date     3989 non-null   object
 1   Kinneret_Level  3989 non-null   object
dtypes: object(2)
memory usage: 62.5+ KB


## Scraping by API from Data.gov Web-Page

In [37]:
import requests as req
import json
import time

def send_call_to_api(n_page):
    """
    The function gets a number of a page, and creates a link, to which we send a request
    if the status code is 200, we return the response as a json.
    
    if we dont get a good response, we send the request again, after 60 seconds.
    """
    url = f'https://data.gov.il/api/3/action/datastore_search?resource_id=cee3ad4a-4e77-4015-8245-52505417d7ea&offset={n_page}'  
    response = req.get(url)
    if response.status_code == 200:
        return json.loads(response.content)
    else:
        print(f'[{response.status_code}]Something is wrong on {n_page}, restarting in 60 seconds...')
        time.sleep(60)
        return send_call_to_api(n_page)
    
def load_json_database(file_path):
    data = pd.read_json(file_path)
    data.to_csv('data_as_csv.csv')
    

def save_data(data, json_name):
    # Loads previous database file
    with open(json_name,'r') as data_json:
         json_data = json.load(data_json)
    
    # adds the new data to the existing file.
    with open(json_name,'w') as new_update:
        json.dump(data,new_update,indent=4)
    return len(json_data)

# This will scrape the data, and save it in database.json
database = []
max_number = 1447400 # Total data items
start_number = 0
done = 0 # The amount of the pages we aleady done crawling
# max_number = 300
for page_num in range(start_number,max_number,100):
    done =+ page_num # adding the number of the current page.
    data_from_requests = send_call_to_api(page_num)['result']['records']
    for record in data_from_requests:
        database.append(record)
    if (done % 10000) == 0: # after each time we get the a number of a page which devides by 10000, we save the data.
        print(f'Done: {done}/{max_number}') # Printing to the console how much we have done
        save_data(database,'database.json') # Saving the data into the json file.
        
print('Scraping has ended')
load_json_database('database.json')
print('data saving has ended')

Done: 0/1447400
[503]Something is wrong on 10000, restarting in 60 seconds...
Done: 10000/1447400
[503]Something is wrong on 20000, restarting in 60 seconds...
Done: 20000/1447400
[503]Something is wrong on 30000, restarting in 60 seconds...
Done: 30000/1447400
[503]Something is wrong on 40000, restarting in 60 seconds...
Done: 40000/1447400
[503]Something is wrong on 50000, restarting in 60 seconds...
Done: 50000/1447400
[503]Something is wrong on 60000, restarting in 60 seconds...
Done: 60000/1447400
[503]Something is wrong on 70000, restarting in 60 seconds...
Done: 70000/1447400
[503]Something is wrong on 80000, restarting in 60 seconds...
Done: 80000/1447400
[503]Something is wrong on 90000, restarting in 60 seconds...
Done: 90000/1447400
[503]Something is wrong on 100000, restarting in 60 seconds...
Done: 100000/1447400
Done: 110000/1447400
Done: 120000/1447400
Done: 130000/1447400
Done: 140000/1447400
Done: 150000/1447400
Done: 160000/1447400
Done: 170000/1447400
Done: 180000/14

In [94]:
daily_Data_df = pd.read_csv("data_as_csv.csv")

In [95]:
daily_Data_df.head()

,Unnamed: 0,_id,stn_num,time_obs,tmp_air_max,tmp_air_min,tmp_grass_min,evp_cls_a_12,evp_cls_a_18,evp_cls_a_06_next,...,rpr_hail,rpr_snow,rpr_frost,rpr_dew,rpr_fog,rpr_mist,rpr_thunder,rpr_lightening,rpr_sand_storm,rpr_gale
0,0,1,9029,1978-04-01T00:00:00,232.0,97.0,70.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,9029,1978-04-02T00:00:00,260.0,92.0,60.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,3,9029,1978-04-03T00:00:00,242.0,138.0,105.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,4,9029,1978-04-04T00:00:00,306.0,100.0,65.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5,9029,1978-04-05T00:00:00,245.0,160.0,148.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
